In [4]:
!python --version

Python 3.9.6


# Genetic algorithm implementation

In [5]:
from typing import List, Optional, Tuple
import random
from random import uniform, randrange

In [6]:
# genetic algorithm parameters
POPULATION_SIZE: int = 100
GENOME_LENGHT: int = 10
VALUES_RANGE: Tuple[float, float] = (-3.0, 3.0)
CROSSOVER_PERCENTAGE: float = 0.3
SURVIVORS_PERCENTAGE: float = 0.1
MUTATION_PROBABILITY: float = 0.1

```
class Gene {
  public Float value;
  public Gene(Float value) { this.value = value; }
  public static Gene get_random(Pair<Float, Float> values_range) {
    random_value = RandomFloat(values_range.getValue0(), values_range.getValue1();
    return new Gene(random_value);}}
```

In [7]:
class Gene:  # == 'trait'/'feature'
    def __init__(self, value: float):
        self.value: float = value
    @classmethod
    def get_random(cls):
        random_value = uniform(VALUES_RANGE[0], VALUES_RANGE[1])
        return cls(random_value)

In [9]:
# class Genome {
#   public ArrayList<Gene> genes;
#   public Genome(ArrayList<Gene> genes) { this.genes = genes; }}

class Genome:  # == 'set of traits/features'
    def __init__(self, genes: List[Gene]):
        self.genes: List[Gene] = genes
    @classmethod
    def get_random(cls):
        genes: List[Gene] = []
        for gene_index in range(GENOME_LENGHT):
            genes.append(Gene.get_random())
        return cls(genes)
    def crossover(self, other_genome):
        result_genes: List[Gene] = []
        for gene_index in range(len(self.genes)):
            if uniform(0.0, 1.0) < 0.5:
                result_genes.append(self.genes[gene_index])
            else:
                result_genes.append(other_genome[gene_index])
        return Genome(result_genes)
    def mutate(self):
        if uniform(0.0, 1.0) < MUTATION_PROBABILITY:
            mutating_gene_index = randrange(len(self.genes))
            self.genes[mutating_gene_index] = uniform(VALUES_RANGE[0], VALUES_RANGE[1])

In [10]:
# class Individual {
#   public Genome genome;
#   public Float score;
#   public Individual(Genome genome) { this.genome = genome }}

class Individual:  # 'genome representative'
    def __init__(self, genome: Genome):
        self.genome: Genome = genome
        self.score: Optional[float] = None
    @classmethod
    def get_random(cls):
        genome = Genome.get_random()
        return cls(genome)

In [11]:
# class Population {
#   public ArrayList<Individual> individuals;
#   public Population(ArrayList<Individual> individuals) { this.individuals = individuals }}

class Population:  # 'set of Individuals'
    def __init__(self, individuals: List[Individual]):
        self.individuals: List[Individual] = individuals
    def get_the_best(self) -> Individual:
        return self.individuals.sort(key=lambda individual:individual.score)[0]
    def get_top_scored(self, percentage: float) -> List[Individual]:
        n_best: int = (len(self.individuals) * percentage).ceil()
        return self.individuals[:n_best]

In [12]:
def generate_init_population():
    generated_individuals: List[Individual] = []
    for individual_index in range(POPULATION_SIZE):
        generated_individuals.append(Individual.get_random())
    return Population(generated_individuals)

In [13]:
def generate_children(individuals: List[Individual], children_number: int, mutation_probability: float) -> List[Individual]:
    children: List[Individual] = []
    for child_index in range(children_number):
        parent1: Individual = random.choice(individuals)
        parent2: Individual = random.choice(individuals.copy().remove(parent1))
        new_genome = parent1.genome.crossover(parent2.genome)
        new_genome.mutate()
        children.append(Individual(new_genome))
    return children

In [14]:
class GeneticAlgProblemSolverAbstractClass:
    def __init__(self, target_score: float, iterations_limit: int):
        self.target_score: float = target_score
        self.iterations_limit: int = iterations_limit
        # generate initial population
        self.current_population: Population = generate_init_population()
            
    # THIS IS THE GENETIC ALGORITHM
    def solve(self) -> Individual:
        # rate individuals
        self.rate_individuals()
        iteration: int = 0
        # if stop condition is met - return the best individual from current_population
        while current_population.get_the_best().score < self.target_score and iteration < self.iterations_limit:
            # choose individuals for crossover
            crossover_individuals: List[Individual] = current_population.get_top_scored(CROSSOVER_PERCENTAGE)
            # get the best individuals from current_population
            survivors: List[Individual] = current_population.get_top_scored(SURVIVORS_PERCENTAGE)
            # generate children by crossover and mutation
            children: List[Individual] = generate_children(crossover_individuals, POPULATION_SIZE - len(survivors))
            # create new population as children plus the best individuals from current_population
            self.current_population = Population(children + survivors)
            # rate individuals
            self.rate_individuals()
        return current_population.get_the_best()
    
    def rate_individuals(self):
        for individual in self.current_population.individuals:
            if individual.score is None:  # if it's not a survivor (avoid rating multiple times)
                individual.score = self.rate_one_individual(individual)
                
    def rate_one_individual(self, individual) -> float:
        # override this method:
        # 1. Interpret individual's genome as potential solution
        # 2. Confront it with the problem
        # 3. Return score for this solution/individual/genome
        pass

# Solving problem